In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import schedule
import time
import croniter
import json
import urllib3
import re
import gspread
import google.oauth2.credentials
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import gspread_dataframe as gsdf
http = urllib3.PoolManager()

def update_votd():
    # This below code will update VOTD data
    # suppress InsecureRequestWarning
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # set up authentication with Google Sheets API
    scope = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
    creds = ServiceAccountCredentials.from_json_keyfile_name('C:\\Users\\Jigar_PAtel\\votdkeys.json', scope)
    client = gspread.authorize(creds)

    # open the Google Sheet you want to update
    sheet = client.open('Tableau_VOTD').sheet1
    # get the first workbook URL from the sheet
    first_workbook_url = sheet.cell(2, 13).value

    # set up parameters for API call
    count = 100
    votd_list = []

    # make API call and append data to list
    votd = json.loads(http.request('GET', f"https://public.tableau.com/api/gallery?page=0&count={count}&galleryType=viz-of-the-day&language=en-us").data)
    votd_list.extend(votd['items'])

    index = None
    for i, item in enumerate(votd_list):
        if item['workbookRepoUrl'] == first_workbook_url:
            index = i
            break

    if index is not None and index != 0:
        # first_workbook_url not found, insert all data
        votd_df = pd.json_normalize(votd_list[:index], max_level=0)
        df = pd.concat([votd_df.drop('workbook', axis=1), pd.json_normalize(votd_df['workbook'].dropna())], axis=1)
        # convert the data to a list of lists
        data = df.values.tolist()
        # loop through the list and convert each value to a string
        def convert_to_strings(data):
            # loop through the list and convert each value to a string
            for i in range(len(data)):
                for j in range(len(data[i])):
                    data[i][j] = str(data[i][j])
            return data
        data1 = convert_to_strings(data)
        row_count = len(data1)
        # specify the row to start inserting the data
        start_row = 2
        # loop through the list and insert each value into the next row
        for i, value in enumerate(data1):
            row = start_row + i
            sheet.insert_row(value, index=row)
        log_sheet = client.open('Tableau_VOTD').worksheet('Update_Log')
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_sheet.append_row([timestamp, row_count])
    else:
        log_sheet = client.open('Tableau_VOTD').worksheet('Update_Log')
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_sheet.append_row([timestamp, 0])
update_votd()        
#schedule.every(1).minutes.do(update_votd) # run the job function every 1 minute
#schedule.every().hour.do(update_votd) # run the job function every hour
#schedule.every().day.at("10:30").do(update_votd) # run the job function every day at 10:30 AM



# importing necessary libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import schedule
import time
import json
import urllib3
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import gspread_dataframe as gsdf
http = urllib3.PoolManager()
# This below code will update VOTD data
# suppress InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# set up authentication with Google Sheets API
scope = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
creds = ServiceAccountCredentials.from_json_keyfile_name('C:\\Users\\Jigar_PAtel\\votdkeys.json', scope)
client = gspread.authorize(creds)

# open the Google Sheet you want to update
sheet = client.open('Tableau_VOTD').sheet1
# get the first workbook URL from the sheet
first_workbook_url = sheet.cell(2, 13).value

# set up parameters for API call
count = 100
votd_list = []

# make API call and append data to list
votd = json.loads(http.request('GET', f"https://public.tableau.com/api/gallery?page=0&count={count}&galleryType=viz-of-the-day&language=en-us").data)
votd_list.extend(votd['items'])

index = None
for i, item in enumerate(votd_list):
    if item['workbookRepoUrl'] == first_workbook_url:
        index = i
        break

if index is not None and index != 0:
    # first_workbook_url not found, insert all data
    votd_df = pd.json_normalize(votd_list[:index], max_level=0)
    df = pd.concat([votd_df.drop('workbook', axis=1), pd.json_normalize(votd_df['workbook'].dropna())], axis=1)
    # convert the data to a list of lists
    data = df.values.tolist()
    # loop through the list and convert each value to a string
    def convert_to_strings(data):
        # loop through the list and convert each value to a string
        for i in range(len(data)):
            for j in range(len(data[i])):
                data[i][j] = str(data[i][j])
        return data
    data1 = convert_to_strings(data)
    row_count = len(data1)
    # specify the row to start inserting the data
    start_row = 2
    # loop through the list and insert each value into the next row
    for i, value in enumerate(data1):
        row = start_row + i
        sheet.insert_row(value, index=row)
    log_sheet = client.open('Tableau_VOTD').worksheet('Update_Log')
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_sheet.append_row([timestamp, row_count])
else:
    log_sheet = client.open('Tableau_VOTD').worksheet('Update_Log')
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_sheet.append_row([timestamp, 0])



# specify the range of cells to check (e.g. A2:D10)
cell_range = "A2:AE2"

# get the values of the cells in the range
cell_values = sheet.range(cell_range)

# loop through the values and check the data type of each value
for cell in cell_values:
    if type(cell.value) == str:
        print(f"Cell {cell.row}, {cell.col} contains a string: {cell.value}")
    elif type(cell.value) == int:
        print(f"Cell {cell.row}, {cell.col} contains an integer: {cell.value}")
    elif type(cell.value) == float:
        print(f"Cell {cell.row}, {cell.col} contains a float: {cell.value}")
    else:
        print(f"Cell {cell.row}, {cell.col} contains a value of an unknown type: {cell.value}")


## this code will get all VOTD Data and store it into Google sheet.

### set up authentication with Google Sheets API
scope = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
creds = ServiceAccountCredentials.from_json_keyfile_name('C:\\Users\\Jigar_PAtel\\votdkeys.json', scope)
client = gspread.authorize(creds)

### open the Google Sheet you want to update
sheet = client.open('Tableau_VOTD').sheet1

### set up parameters for API call
page = 0
count = 100
votd_list = []

while True:
    votd = json.loads(http.request('GET', f"https://public.tableau.com/api/gallery?page={page}&count={count}&galleryType=viz-of-the-day&language=en-us").data)
    votd_list.extend(votd['items'])
    if len(votd['items']) < count:
        break
    page += 1

### normalize the new data and select columns to keep
votd_normal = pd.json_normalize(votd_list, max_level=0)
df = pd.concat([votd_normal.drop('workbook', axis=1), pd.json_normalize(votd_normal['workbook'].dropna())], axis=1)
### save updated data to Google Sheet
gsdf.set_with_dataframe(sheet, df)

### open the Update Log sheet
update_log_sheet = client.open('Tableau_VOTD').worksheet('Update_Log')

### get the current time
current_time = datetime.now().strftime("%m/%d/%Y %H:%M:%S")

### add a new row to the Update Log sheet
update_log_sheet.append_row([current_time, 'VOTD data updated'])
